<a href="https://colab.research.google.com/github/ashuonnet/ailearn/blob/main/Resume_Skills_Extractors_Spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy_transformers
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.6/194.6 KB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 104.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 111.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.5 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.4
    Uninstalling spacy-3.4.4:
      Successfully uninstalled spacy-3.4.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.4.1 requires spacy<3.5.0,>=3.4.0, but you have spacy 3.5.0 which is incompatible.

In [ ]:
import json

In [ ]:
!nvidia-smi

Tue Feb 28 14:24:39 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    29W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!rm -r skill_extractor
!git clone https://github.com/ashuonnet/skill_extractor

rm: cannot remove 'skill_extractor': No such file or directory
Cloning into 'skill_extractor'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 28 (delta 8), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (28/28), 503.33 KiB | 1.77 MiB/s, done.


In [ ]:
!python -m spacy init fill-config /content/skill_extractor/data/train/base_config.cfg /content/skill_extractor/data/train/config.cfg

2023-02-28 14:24:46.277997: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-28 14:24:47.152402: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-28 14:24:47.152529: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-28 14:24:47.152550: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

## Helper Functions for converting annotations to spacy format. 

In [ ]:
import json
import re

# JSON formatting functions
def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
    training_data = []
    lines=[]
    with open(dataturks_JSON_FilePath, 'r') as f:
        lines = f.readlines()

    for line in lines:
        data = json.loads(line)
        text = data['content'].replace("\n", " ")
        entities = []
        data_annotations = data['annotation']
        if data_annotations is not None:
            for annotation in data_annotations:
                #only a single point in text annotation.
                point = annotation['points'][0]
                labels = annotation['label']
                # handle both list of labels or a single label.
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    point_start = point['start']
                    point_end = point['end']
                    point_text = point['text']

                    lstrip_diff = len(point_text) - len(point_text.lstrip())
                    rstrip_diff = len(point_text) - len(point_text.rstrip())
                    if lstrip_diff != 0:
                        point_start = point_start + lstrip_diff
                    if rstrip_diff != 0:
                        point_end = point_end - rstrip_diff
                    entities.append((point_start, point_end + 1 , label))
        training_data.append((text, {"entities" : entities}))
    return training_data

def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])
    return cleaned_data

In [ ]:
#cv_data = json.load(open('/content/skill_extractor/data/train/resumes_1.json'))
cv_data = trim_entity_spans(convert_dataturks_to_spacy('/content/skill_extractor/data/train/resumes_1.json'))

In [ ]:
cv_data[1]

['Afreen Jamadar Active member of IIIT Committee in Third year  Sangli, Maharashtra - Email me on Indeed: indeed.com/r/Afreen-Jamadar/8baf379b705e37c6  I wish to use my knowledge, skills and conceptual understanding to create excellent team environments and work consistently achieving organization objectives believes in taking initiative and work to excellence in my work.  WORK EXPERIENCE  Active member of IIIT Committee in Third year  Cisco Networking -  Kanpur, Uttar Pradesh  organized by Techkriti IIT Kanpur and Azure Skynet. PERSONALLITY TRAITS: • Quick learning ability • hard working  EDUCATION  PG-DAC  CDAC ACTS  2017  Bachelor of Engg in Information Technology  Shivaji University Kolhapur -  Kolhapur, Maharashtra  2016  SKILLS  Database (Less than 1 year), HTML (Less than 1 year), Linux. (Less than 1 year), MICROSOFT ACCESS (Less than 1 year), MICROSOFT WINDOWS (Less than 1 year)  ADDITIONAL INFORMATION  TECHNICAL SKILLS:  • Programming Languages: C, C++, Java, .net, php. • Web 

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm  

In [ ]:
spacy.__version__

'3.5.0'

In [ ]:
def get_spacy_doc(file, data):
    nlp = spacy.blank('en')
    db = DocBin()
        
    for text, annot in tqdm(data):
        doc = nlp.make_doc(text)
        annot = annot['entities']
    
        ents = []
        entity_indices = []
            
        for start, end, label in annot:
            skip_entity = False
            for idx in range(start, end):
                if idx in entity_indices:
                    skip_entity = True
                    break
            if skip_entity == True:
                continue 
                    
            entity_indices = entity_indices + list(range(start, end))  
                
            try:
                span = doc.char_span(start, end,label=label,alignment_mode = 'strict')
            except:
                continue

            if span is None:
                err_data = str([start, end]) + " " + str(text) + "\n "
                err_data = str(err_data.encode("UTF-8"))
                file.write(err_data)
            else:
                ents.append(span)

        try:
            doc.ents = ents
            db.add(doc)
        except:
            pass 
    return db

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.3)

In [ ]:
len(train),len(test)

(154, 66)

In [ ]:
file = open('error.txt','w')

db = get_spacy_doc(file, train)
db.to_disk('/content/skill_extractor/data/train/train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('/content/skill_extractor/data/train/test_data.spacy')

file.close()

100%|██████████| 66/66 [00:00<00:00, 70.83it/s]


In [ ]:
len(db.tokens)

66

In [ ]:
#Monkey patch for below issue, but works fine. Error reported for shell cmd
############################################################################
# NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968
############################################################################
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding


In [ ]:
!python -m spacy train /content/skill_extractor/data/train/config.cfg --output ./output --paths.train /content/skill_extractor/data/train/train_data.spacy --paths.dev /content/skill_extractor/data/train/test_data.spacy --gpu-id 0

2023-02-28 14:25:13.093654: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-28 14:25:13.093758: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-28 14:25:13.093778: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[

In [ ]:
!zip -r /content/model-best.zip /content/output/model-best

In [ ]:
nlp = spacy.load('./output/model-best')

In [ ]:
txt = 'My name is Kashmira sharma. I work as a dentist. I completed my degree in Bachelors of Dental Surgery from Government dental College Mumbai. SKILLs : Dental Works'

In [ ]:
doc = nlp(txt)
for ent in doc.ents:
    print(ent.label_.upper(),"  =>  ", ent.text)

In [ ]:
test_resume = 'Ram Krishna Chauhan  '\
'Email ID: testemail@gmail.com '\
'Phone : 09963987441 '\
'Job Objective '\
'To work in the most challenging position with an organization that provides ample opportunities to learn, utilize my skills and abilities with extreme '\
'professionalism and to be highly resourceful, innovative and flexible whenever required. '\
'Experience Summary '\
'6 years of experience in C, Linux, Networking. '\
'Have experience in Packet Forwarding support on Cisco 12000 series router. '\
'Hands on experience in working with Broadcom chipset ARAD+. '\
'Expertise in Handling Customer found defects and fixing them at high priority. '\
'Good knowledge in protocols like IPV4, L3VPN, NAT, L2VPN. '\
'Good understanding of TCAM, Cisco IOS-XR architecture. '\
'Basic Hands on experience in System programming. '\
'Technical skills: '\
'Language '\
'C, Micro Code. '\
'Networking Technologies '\
'IPv4, L3VPN, L2VPN, MPLS, NAT '\
'OS '\
'IOS-XR, Linux '\
'Academic Chronicle '\
'B.Tech. in Electronics & Communication Engineering with 79% '\
'V.R Siddhartha Engineering College - Nagarjuna University. '\
'Intermediate during 2005-2007 with 96% '\
'Sri Engineering Educational Institute - State Board. '\
'S.S.C during 2004-2005 with 86% '\
'St Johnes public school - State Board. '\
'Work Experience: '\
'Gigamon : Feb 2017 - till date '\
'Designation : Senior Engineer. '\
'CODC, HCL Technologies : May 2011 - Feb 2017 '\
'Designation : Lead Engineer. '\
'Project - H Series Devices Development/Sustenance (Gigamon) '\
'Hardware : Gigamon H Series Devices '\
'Software : C language, Linux '\
'Tools : GIT, JIRA '\
'Duration : Feb 2017 to till date '\
'Description : H Series delivers performance and intelligence in each of its Visibility Fabric nodes, with port density and speeds that scales from 1 Gb to '\
'100 Gb. The Visibility Fabric is able to replicate, filter and selectively forward network traffic to monitoring, management and security tools. '\
'Role '\
'Handling both externally found and internally found defects. '\
'Project - ARWEN LC DEVELOPMENT (HCL - CODC) '\
'Hardware : ARWEN LC '\
'Software : C language, Cisco IOS-XR '\
'Tools : ACME, DDTS '\
'Duration : 1 year 3 months '\
'Description : Arwen is internal codename for Scapa packet card based on Broadcom’s Dune family of ARAD+ packet processor to provide combined packet and OTN  '\
'switching functionality. '\
'Role '\
'XR Bring-up of GRIMA FPGA for external stats support. '\
'Integrated multiple feature stats for Arwen LC. '\
'Project - CRS -CGN Development/Sustenance (HCL - CODC) '\
'Hardware : CRS - CGSE + LC. '\
'Software : C language, Cisco IOS-XR '\
'Tools : ACME, DDTS '\
'Duration : 1 year '\
'Description : Carrier Grade Network Address Translation (CGN) is a large scale NAT that is capable of providing private IPv4 to public IPv4 address  '\
'translation in the order of millions of translations to support a large number of subscribers, and at least 10 Gbps full-duplex bandwidth throughput. '\
'Role '\
'Worked on Port-preservation feature and bug fixing. '\
'3. Project - GSR FORWARDING Development/Sustenance (HCL - CODC) '\
'Hardware : C12000 '\
'Software : C language, Cisco IOS, Cisco IOS-XR '\
'Tools : Clear Case, ACME, DDTS '\
'Duration : 2 year 9 months '\
'Description : Gigabit Switch Router (GSR) is a router designed and manufactured by Cisco Systems to provide IP and MPLS services. GSR has a distributed '\
'architecture. Control plane lies on RP (Route processor) and data plane lies on LC\'s (Line card). GSR uses distributed CEF for performing packet '\
'forwarding. GSR has different flavors of Line-cards E0 to E6 and uses IOS - XR (Inter Networking operating system) as the Operating System. '\
'CLI implementation to reload the LC, once pre-cam failure count reaches threshold value. '
'Role '\
'Support for TCAM, Extended Features, L3 packet forwarding on E3 and E5 LC’s. '\
'Handling both externally and internally found defects on E3 and E5. '\
'Experience in Microcode debugging and bug fixing. '\
'Personal details '\
'Father’s Name '\
'P. S. Murthy '\
'Date of Birth ' 
'25-08-1992 '\
'Marital Status '\
'Single '\
'Languages known '\
'English, Telugu, Hindi, Tamil '\
'E-Mail '\
'testemail@gmail.com '\
'Contact Address '\
'23, 23-4, '\
'Nagarjuna teample Road, '\
'Kakinada, '\
'Vijayawada - 520012 '\
'Declaration: '\
'I hereby declare that the above written particulars are true to the best of my knowledge and belief. '\
'Place: Chennai Regards '\
'Date: Ram Krishna Chauhan '

In [ ]:
doc = nlp(test_resume)
for ent in doc.ents:
    print(ent.label_.upper(),"  =>  ", ent.text)

In [ ]:
#NLP = spacy.load('./output/model-last')